In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df1 = pd.read_csv("../data1.csv")
areas = ['r' + str(i) for i in range(1, 35)] + ['c1', 'c2', 'outside']
truth1 = df1[areas]

In [20]:
# display(truth1.head())
# display(truth1.shift().head())

### deltas1 stores the change in people in each room per time period.
deltas1 = (truth1 - truth1.shift()).dropna()
deltas1.head()

# Index 200 is interesting - indicates a potential move from c2 to r27, or r13 to r27. 
moves = {K:V for K, V in dict(deltas1.iloc[200]).items() if V != 0}
print(moves)

{'r12': 1.0, 'r13': -1.0, 'r27': 1.0, 'c1': 1.0, 'c2': -2.0}


In [11]:
candidateNeighbours = {
    'r1':[],
    'r2':[],
    'r3':[],
    'r4':[],
    'r5':[],
    'r6':[],
    'r7':[],
    'r8':[],
    'r9':[],
    'r10':[],
    'r11':[],
    'r12':[],
    'r13':[],
    'r14':[],
    'r15':[],
    'r16':[],
    'r17':[],
    'r18':[],
    'r19':[],
    'r20':[],
    'r21':[],
    'r22':[],
    'r23':[],
    'r24':[],
    'r25':[],
    'r26':[],
    'r27':[],
    'r28':[],
    'r29':[],
    'r30':[],
    'r31':[],
    'r32':[],
    'r33':[],
    'r34':[],
    'c1':[],
    'c2':[],
    'outside':[],
}

['r1',
 'r2',
 'r3',
 'r4',
 'r5',
 'r6',
 'r7',
 'r8',
 'r9',
 'r10',
 'r11',
 'r12',
 'r13',
 'r14',
 'r15',
 'r16',
 'r17',
 'r18',
 'r19',
 'r20',
 'r21',
 'r22',
 'r23',
 'r24',
 'r25',
 'r26',
 'r27',
 'r28',
 'r29',
 'r30',
 'r31',
 'r32',
 'r33',
 'r34',
 'c1',
 'c2',
 'outside']